In [1]:

import pandas as pd
import mysql.connector

In [2]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',   # change this
    database='placement_db'
)



In [4]:
# Read data into Pandas
df = pd.read_sql("SELECT * FROM students", conn)
#print(df.head())
print(df.describe())
Q1 = df['CGPA'].quantile(0.25)
Q3 = df['CGPA'].quantile(0.75)
IQR = Q3 - Q1

                IQ  Prev_Sem_Result         CGPA  Academic_Performance  \
count  9751.000000      9751.000000  9751.000000           9751.000000   
mean     99.479438         7.476841     7.465179              5.546200   
std      15.069030         1.417550     1.426456              2.870628   
min      41.000000         5.000000     4.540000              1.000000   
25%      89.000000         6.250000     6.260000              3.000000   
50%      99.000000         7.500000     7.490000              6.000000   
75%     110.000000         8.700000     8.690000              8.000000   
max     158.000000        10.000000    10.000000             10.000000   

       Extra_Curricular_Score  Communication_Skills  Projects_Completed  
count             9751.000000           9751.000000         9751.000000  
mean                 4.967798              5.563737            2.509896  
std                  3.161108              2.900506            1.715773  
min                  0.000000        

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1152\769757907.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM students", conn)


In [5]:
# Remove outliers
df = df[(df['CGPA'] >= Q1 - 1.5 * IQR) & (df['CGPA'] <= Q3 + 1.5 * IQR)]
# Convert to string first



In [6]:
print(df['Internship_Experience'])

0        no
1        no
2        no
3       yes
4        no
       ... 
9746     no
9747     no
9748    yes
9749     no
9750     no
Name: Internship_Experience, Length: 9751, dtype: object


In [7]:

df['Internship_Experience']= df['Internship_Experience'].map({'yes':1,'no':0})

print(df['Internship_Experience'])

0       0
1       0
2       0
3       1
4       0
       ..
9746    0
9747    0
9748    1
9749    0
9750    0
Name: Internship_Experience, Length: 9751, dtype: int64


In [8]:

df['Placement']= df['Placement'].map({'yes':1,'no':0})
print(df['Placement'])


0       0
1       0
2       0
3       0
4       0
       ..
9746    1
9747    0
9748    0
9749    0
9750    0
Name: Placement, Length: 9751, dtype: int64


In [9]:
df.drop(columns = 'College_ID',inplace = True)

In [10]:
print(df.head())
from sklearn.model_selection import train_test_split



    IQ  Prev_Sem_Result  CGPA  Academic_Performance  Internship_Experience  \
0  107             6.61  6.28                     8                      0   
1   97             5.52  5.37                     8                      0   
2  109             5.36  5.83                     9                      0   
3  122             5.47  5.75                     6                      1   
4   96             7.91  7.69                     7                      0   

   Extra_Curricular_Score  Communication_Skills  Projects_Completed  Placement  
0                       8                     8                   4          0  
1                       7                     8                   0          0  
2                       3                     1                   1          0  
3                       1                     6                   1          0  
4                       8                    10                   2          0  


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x = df.drop('Placement',axis = 1)

y = df['Placement']

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [14]:
print(x_train)


       IQ  Prev_Sem_Result  CGPA  Academic_Performance  Internship_Experience  \
7771   91             9.22  8.84                     9                      1   
4793   80             5.53  5.23                     8                      0   
4639   91             6.48  6.30                     4                      0   
8594   86             7.76  7.44                    10                      1   
8783   89             9.71  9.72                     1                      0   
...   ...              ...   ...                   ...                    ...   
5734   90             6.46  6.46                     7                      0   
5191   96             5.93  5.73                     8                      1   
5390   90             9.57  9.66                     8                      1   
860   103             8.39  8.32                     8                      1   
7270  118             6.65  6.35                     9                      0   

      Extra_Curricular_Scor

In [15]:
print(y_test)

4535    0
2516    1
1606    0
9569    0
8948    0
       ..
3269    1
2755    0
2794    0
2997    0
7202    0
Name: Placement, Length: 1951, dtype: int64


In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()


In [17]:
print(x_train.dtypes)


IQ                          int64
Prev_Sem_Result           float64
CGPA                      float64
Academic_Performance        int64
Internship_Experience       int64
Extra_Curricular_Score      int64
Communication_Skills        int64
Projects_Completed          int64
dtype: object


In [18]:
print(x_train.isnull().sum())


IQ                        0
Prev_Sem_Result           0
CGPA                      0
Academic_Performance      0
Internship_Experience     0
Extra_Curricular_Score    0
Communication_Skills      0
Projects_Completed        0
dtype: int64


In [19]:
model.fit(x_train, y_train)


d:\OneDrive - BAAZAR STYLE RETAIL PRIVATE LIMITED\Desktop\StudentAttendanceSystem\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
y_pred = model.predict(x_test)

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.910814966683752
Confusion Matrix:
 [[1565   56]
 [ 118  212]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95      1621
           1       0.79      0.64      0.71       330

    accuracy                           0.91      1951
   macro avg       0.86      0.80      0.83      1951
weighted avg       0.91      0.91      0.91      1951



In [22]:
model = LogisticRegression(class_weight='balanced',max_iter=1000)

In [23]:
model.fit(x_train,y_train)
{}

{}

In [24]:
y_pred=model.predict(x_test)

In [25]:
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,classification_report
print("Accuracy is: ", accuracy_score(y_pred,y_test))
print("recall score is:", classification_report(y_pred, y_test))

Accuracy is:  0.8559712967708867
recall score is:               precision    recall  f1-score   support

           0       0.85      0.98      0.91      1408
           1       0.90      0.55      0.68       543

    accuracy                           0.86      1951
   macro avg       0.87      0.76      0.79      1951
weighted avg       0.86      0.86      0.84      1951



In [26]:
from sklearn.model_selection import cross_val_score

In [27]:
scores = cross_val_score(model,x,y,cv =5, scoring='recall')

In [28]:
print(scores)
print(scores.mean())

[0.8952381  0.87936508 0.87301587 0.85079365 0.88253968]
0.8761904761904763


In [29]:
import joblib

joblib.dump(model, 'placement_model.pkl')




['placement_model.pkl']

In [30]:
model = joblib.load('placement_model.pkl')

In [31]:
new_data = [[110, 7.5, 8.0, 0, 2, 0, 0, 6]]  # replace with your own

# Predict placement
prediction = model.predict(new_data)
print("Placed" if prediction[0] == 1 else "Not Placed")

Not Placed


d:\OneDrive - BAAZAR STYLE RETAIL PRIVATE LIMITED\Desktop\StudentAttendanceSystem\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [32]:
importance = model.coef_[0]
for col, imp in zip(x_train.columns, importance):
    print(f"{col}: {imp:.3f}")


IQ: 0.111
Prev_Sem_Result: 0.046
CGPA: 1.303
Academic_Performance: -0.003
Internship_Experience: 0.007
Extra_Curricular_Score: -0.014
Communication_Skills: 0.646
Projects_Completed: 0.679


In [33]:
import streamlit as st
import joblib
import numpy as np

# Load model
model = joblib.load('placement_model.pkl')

# Page config
st.set_page_config(page_title="Smart Student Placement Predictor", layout="centered")

# Page title
st.markdown(
    "<h1 style='text-align: center; color: #4B8BBE;'>🎓 Smart Student Placement Predictor</h1>",
    unsafe_allow_html=True
)

# Instructions
st.markdown(
    """
    <div style='text-align: center; font-size: 16px; color: #6c757d;'>
    Enter student details below to check if they are likely to be placed.
    </div><br>
    """,
    unsafe_allow_html=True
)

# Input form
with st.form("placement_form"):
    col1, col2 = st.columns(2)

    with col1:
        iq = st.number_input("🧠 IQ Score", min_value=50.0, max_value=200.0, step=1.0)
        prev_sem = st.number_input("📊 Previous Semester Result (out of 10)", min_value=0.0, max_value=10.0, step=0.1)
        cgpa = st.number_input("🎯 CGPA", min_value=0.0, max_value=10.0, step=0.1)
        academic = st.number_input("📘 Academic Performance (Score)", min_value=0, max_value=10, step=1)

    with col2:
        internship = st.selectbox("💼 Internship Experience", options=["No", "Yes"])
        extra_curricular = st.number_input("🏅 Extra Curricular Score", min_value=0, max_value=10, step=1)
        communication = st.number_input("🗣️ Communication Skills", min_value=0, max_value=10, step=1)
        projects = st.number_input("📂 Number of Projects", min_value=0, max_value=10, step=1)

    submitted = st.form_submit_button("🔍 Predict Placement")

    if submitted:
        internship_val = 1 if internship == "Yes" else 0
        input_data = [[iq, prev_sem, cgpa, academic, internship_val, extra_curricular, communication, projects]]
        prediction = model.predict(input_data)

        if prediction[0] == 1:
            st.success("🎉 The student is **likely to be placed!**")
        else:
            st.error("⚠️ The student is **not likely to be placed.**")


2025-07-30 12:23:21.379 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:23:21.380 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:23:21.563 
  command:

    streamlit run C:\Users\Administrator\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-30 12:23:21.564 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:23:21.565 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:23:21.565 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:23:21.566 Thread 'MainThread': missing ScriptRunContext! This warni

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
rf_model = RandomForestClassifier(class_weight='balanced',n_estimators=100,random_state=42)

In [36]:
rf_model.fit(x_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [37]:
rf_pred = rf_model.predict(x_test)

In [38]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("accuracy is: ", accuracy_score(y_test,rf_pred))

accuracy is:  0.9994874423372629


In [39]:
print("classfication report is:", classification_report(y_test,rf_pred))

classfication report is:               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1621
           1       1.00      1.00      1.00       330

    accuracy                           1.00      1951
   macro avg       1.00      1.00      1.00      1951
weighted avg       1.00      1.00      1.00      1951



In [49]:
from sklearn.model_selection import cross_val_score
rf_scores = cross_val_score(model,x,y,cv =5,scoring = 'accuracy')

In [50]:
print(rf_scores)

[0.85802153 0.85589744 0.84820513 0.85179487 0.84358974]


In [52]:
import joblib

In [56]:
joblib.dump(rf_model,'rf_placement_model.pkl')

['rf_placement_model.pkl']

In [57]:
# app.py
import streamlit as st
import joblib
import numpy as np
import pandas as pd

# Load models
log_model = joblib.load('placement_model.pkl')
rf_model = joblib.load('rf_placement_model.pkl')

# Set page config
st.set_page_config(page_title="Student Placement Predictor", layout="centered")

st.title("🎓 Smart Student Placement Predictor")
st.markdown("Predict if a student is likely to be placed based on academic and personal attributes.")

# Model selection
model_choice = st.selectbox("Choose Model", ["Logistic Regression", "Random Forest"])

# Input fields
st.subheader("📥 Enter Student Details")

iq = st.number_input("IQ", min_value=50, max_value=200, step=1)
prev_result = st.number_input("Previous Semester Result", min_value=0.0, max_value=10.0, step=0.1)
cgpa = st.number_input("CGPA", min_value=0.0, max_value=10.0, step=0.1)
academic_perf = st.number_input("Academic Performance", min_value=0, max_value=10, step=1)
internship = st.selectbox("Internship Experience", ["No", "Yes"])
extra_curricular = st.number_input("Extra Curricular Score", min_value=0, max_value=10, step=1)
communication = st.number_input("Communication Skills", min_value=0, max_value=10, step=1)
projects = st.number_input("Number of Projects Completed", min_value=0, max_value=10, step=1)

# Convert categorical input
internship_val = 1 if internship == "Yes" else 0

input_data = np.array([[iq, prev_result, cgpa, academic_perf, internship_val, extra_curricular, communication, projects]])

# Predict button
if st.button("🎯 Predict Placement"):
    if model_choice == "Logistic Regression":
        model = log_model
    else:
        model = rf_model

    pred = model.predict(input_data)[0]
    proba = model.predict_proba(input_data)[0][1]

    if pred == 1:
        st.success(f"✅ Student is likely to be **Placed**! \n\n📊 Confidence: `{proba*100:.2f}%`")
    else:
        st.error(f"❌ Student is **Not Likely to be Placed**.\n\n📊 Confidence: `{(1-proba)*100:.2f}%`")

# Optional: Feature Importance for RF
if model_choice == "Random Forest":
    st.subheader("📌 Feature Importance (Random Forest)")
    importance = rf_model.feature_importances_
    features = ['IQ', 'Prev Result', 'CGPA', 'Academic', 'Internship', 'Extra Curricular', 'Communication', 'Projects']
    df_feat = pd.DataFrame({'Feature': features, 'Importance': importance})
    df_feat = df_feat.sort_values(by='Importance', ascending=False)

    st.bar_chart(df_feat.set_index("Feature"))


2025-07-30 12:40:41.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-30 12:40:41.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar